<a href="https://colab.research.google.com/github/eduion/AIOT/blob/main/HW4/HW4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pycaret
!pip install catboost
import pandas as pd
from sklearn.model_selection import train_test_split
from pycaret.classification import *

def simplify_ages(df):
    df['Age'] = df['Age'].fillna(-0.5)
    bins = (-1, 0, 5, 12, 18, 25, 35, 60, 120)
    group_names = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young_Adult', 'Adult', 'Senior']
    categories = pd.cut(df['Age'], bins, labels=group_names)
    df['Age'] = categories
    return df

def simplify_cabins(df):
    df['Cabin'] = df['Cabin'].fillna('N')
    df['Cabin'] = df['Cabin'].apply(lambda x: x[0])
    return df

def simplify_fares(df):
    df['Fare'] = df['Fare'].fillna(-0.5)
    bins = (-1, 0, 8, 15, 31, 1000)
    group_names = ['Unknown', '1_quartile', '2_quartile', '3_quartile', '4_quartile']
    categories = pd.cut(df['Fare'], bins, labels=group_names)
    df['Fare'] = categories
    return df

def format_name(df):
    df['Lname'] = df['Name'].apply(lambda x: x.split(' ')[0])
    df['NamePrefix'] = df['Name'].apply(lambda x: x.split(' ')[1])
    return df

def drop_features(df):
    return df.drop(['Ticket', 'Name', 'Embarked'], axis=1)

def transform_features(df):
    df = simplify_ages(df)
    df = simplify_cabins(df)
    df = simplify_fares(df)
    df = format_name(df)
    df = drop_features(df)
    return df

def rename_features(df):
    df.columns = df.columns.str.replace(r'[^\w]', '_', regex=True)
    return df

# 主程序
if __name__ == "__main__":
    # 載入數據集
    data_train = pd.read_csv('./train.csv')  # 將路徑改為實際數據的位置

    # 轉換特徵
    data_train = transform_features(data_train)
    data_train = rename_features(data_train)

    # 分割數據
    train_data, test_data = train_test_split(data_train.drop(['PassengerId'], axis=1), random_state=100, train_size=0.8)

    # 設定 PyCaret
    clf1 = setup(data=train_data,
                 target='Survived',
                 categorical_features=['Pclass', 'Sex', 'Age', 'Fare', 'Cabin', 'Lname', 'NamePrefix'],
                 session_id=123)

    # 1. 比較模型並選擇最佳模型
    best_model = compare_models(fold=5, sort='Accuracy')

    # 2. 集成模型
    top_models = [create_model(m, fold=5) for m in ['lr', 'knn', 'nb', 'dt', 'rf']]  # 手動選擇模型
    tuned_models = [tune_model(m, fold=5, optimize='Accuracy') for m in top_models]  # 調參
    ensemble_model = blend_models(estimator_list=tuned_models, fold=5)

    # 3. 超參數優化
    optimized_model = tune_model(ensemble_model, fold=5, optimize='Accuracy')

    # 訓練最佳模型並進行預測
    final_model = finalize_model(optimized_model)
    predictions = predict_model(final_model, data=test_data)

    # 輸出最佳模型和預測結果
    print("Best Model:", best_model)
    print("Predictions:")
    print(predictions)



,Description,Value
0,Session id,123
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(712, 10)"
4,Transformed data shape,"(712, 30)"
5,Transformed train set shape,"(498, 30)"
6,Transformed test set shape,"(214, 30)"
7,Numeric features,2
8,Categorical features,7
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.8031,0.8557,0.6782,0.7711,0.7198,0.5692,0.5735,0.1520
lr,Logistic Regression,0.8011,0.8710,0.6993,0.7542,0.7219,0.5679,0.5724,0.2600
knn,K Neighbors Classifier,0.7972,0.8454,0.6519,0.7714,0.7057,0.5530,0.5581,0.2640
et,Extra Trees Classifier,0.7771,0.8290,0.6947,0.7066,0.7001,0.5228,0.5233,0.4520
rf,Random Forest Classifier,0.7691,0.8276,0.6945,0.6948,0.6926,0.5078,0.5099,0.3440
gbc,Gradient Boosting Classifier,0.7671,0.7646,0.7003,0.6865,0.6930,0.5054,0.5058,0.2620
catboost,CatBoost Classifier,0.7511,0.7845,0.6161,0.6935,0.6465,0.4571,0.4630,0.9480
lightgbm,Light Gradient Boosting Machine,0.7489,0.7494,0.7161,0.6501,0.6812,0.4748,0.4767,1.1820
dt,Decision Tree Classifier,0.7449,0.7351,0.6952,0.6521,0.6722,0.4642,0.4655,0.1660
ada,Ada Boost Classifier,0.7408,0.7487,0.6687,0.6652,0.6590,0.4517,0.4582,0.2480


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7900,0.8863,0.6486,0.7500,0.6957,0.5366,0.5399
1,0.8100,0.8873,0.7895,0.7317,0.7595,0.6028,0.6040
2,0.8400,0.9107,0.8421,0.7619,0.8000,0.6672,0.6695
3,0.7475,0.8184,0.5676,0.7000,0.6269,0.4392,0.4447
4,0.8182,0.8524,0.6486,0.8276,0.7273,0.5939,0.6038
Mean,0.8011,0.8710,0.6993,0.7542,0.7219,0.5679,0.5724
Std,0.0313,0.0322,0.1010,0.0422,0.0588,0.0766,0.0759


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8400,0.8782,0.7297,0.8182,0.7714,0.6490,0.6515
1,0.8600,0.9043,0.8421,0.8000,0.8205,0.7059,0.7065
2,0.8500,0.8911,0.8158,0.7949,0.8052,0.6833,0.6834
3,0.7576,0.8337,0.6216,0.6970,0.6571,0.4706,0.4724
4,0.8081,0.8633,0.6486,0.8000,0.7164,0.5738,0.5810
Mean,0.8231,0.8741,0.7316,0.7820,0.7541,0.6165,0.6190
Std,0.0371,0.0244,0.0875,0.0433,0.0602,0.0856,0.0846


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8045,0.8646,0.6133,0.8846,0.7244,0.5805,0.6039


Best Model: RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, positive=False, random_state=123, solver='auto',
                tol=0.0001)
Predictions:
     Pclass     Sex          Age  SibSp  Parch        Fare Cabin  \
205       3  female         Baby      0      1  2_quartile     G   
44        3  female      Student      0      0  1_quartile     N   
821       3    male  Young_Adult      0      0  2_quartile     N   
458       2  female        Adult      0      0  2_quartile     N   
795       2    male        Adult      0      0  2_quartile     N   
..      ...     ...          ...    ...    ...         ...   ...   
247       2  female      Student      0      2  2_quartile     N   
259       2  female        Adult      0      1  3_quartile     N   
353       3    male      Student      1      0  3_quartile     N   
783       3    male      Unknown      1      2  3_quartile     N   
852       3  female        Child      1 